# Technical part

In [1]:
import random
import numpy as np
from scipy import stats

In [2]:
class Profile():
    '''
    Profile object to capture all the links created between profiles by likes
    '''

    def __init__(self, profile_id):
        self.profile_id = profile_id # just a number
        self.likes_given_to = set() # set of profiles who received likes from current profile
        self.likes_received_from = set() # set of profiles who liked current profile
        self.matches = set()
        self.sex = 'X' # will be defined on the like distribution stage

    def find_matches(self):
        '''
        find likes by intersecting the ones who were liked with ones who liked
        '''
        self.matches = self.likes_given_to & self.likes_received_from

    def statistics(self):
        '''
        calculate amount of received likes and amount of likes for this profile
        '''
        return len(self.likes_received_from), len(self.matches)

    def introduce(self):
        '''
        pretty representation of a profile info
        '''
        print(f'I\'m {self.sex} my id is {self.profile_id}')
        print('I like:')
        print(*map(lambda x: x.sex + '_' + str(x.profile_id), self.likes_given_to), sep=', ')
        print('I was liked by:')
        print(*map(lambda x: x.sex + '_' + str(x.profile_id), self.likes_received_from), sep=', ')
        print('I matched with:')
        print(*map(lambda x: x.sex + '_' + str(x.profile_id), self.matches), sep=', ')
        print(f'In total I received {self.statistics()[0]} likes and {self.statistics()[1]} of those matched')

In [3]:
def distribute_likes(population, amount_of_F=500, amount_of_swipes=100, F_to_M_prob = 0.25, M_to_F_prob = 0.25):
    '''
    distribute likes to random profiles of opposite sex
    '''
    for profile in population:
        if profile.profile_id <= amount_of_F:
            profile.sex = 'F'
            profile.likes_given_to = set(random.sample(population[amount_of_F + 1 :], k=int(amount_of_swipes * F_to_M_prob)))
        else:
            profile.sex = 'M'
            profile.likes_given_to = set(random.sample(population[: amount_of_F + 1], k=int(amount_of_swipes * M_to_F_prob)))

In [4]:
def receive_likes(population):
    for profile in population:
        for liked_profile in profile.likes_given_to:
            liked_profile.likes_received_from.add(profile)

In [5]:
def central_tendencies(population):
    for sex in ('F', 'M'):
        print(f'Average amount of likes and matches per {sex} profile')
        print(np.average(np.array([*map(Profile.statistics, filter(lambda x: x.sex == sex, population))]), axis=0))
        print(f'Mode of likes of matches per {sex} profile')
        print(stats.mode(np.array([*map(Profile.statistics, filter(lambda x: x.sex == sex, population))]), axis=0))
        print()

# Case 1

F population = M population, P(F to give a like) = P(M to give a like)

Let's calculate a probability of receiving a like given the size of population. Average amount of likes received from 1 person is essentialy a probability of receiveng a like:

$
Probability\ to\ receive\ a\ like\ given\ that\ amount\ of\ swipes\ is\ limited = Average\ amount\ of\ likes\ received\ from\ 1\ preson = \frac{Amount\ of\ swipes}{Amount\ of\ people\ who\ can\ give\ a\ like(opposite\ sex)} * Probability\ of\ giving\ a\ like(same\ sex)
$

$
Average\ amount\ of\ likes\ received\ from\ 1\ F = Average\ amount\ of\ likes\ received\ from\ 1\ M = \frac{100}{500} * 0.25 = 0.05
$

In other words: one person gives on average 0.05 likes to each person of opposite sex. Given that we have 500 people of each sex we have $Average\ amount\ of\ likes\ received = 0.05 * 500 = 25$ 

Let's calculate a probability of giving a like given the size of population. Average amount of likes given by 1 person is essentialy a probability of giving a like:

$
Probability\ to\ give\ a\ like\ given\ that\ amount\ of\ swipes\ is\ limited = Average\ amount\ of\ likes\ given\ to\ 1\ preson =\frac{Amount\ of\ swipes}{Amount\ of\ people\ who\ can\ give\ a\ like(same\ sex)} * Probability\ of\ giving\ a\ like(opposite\ sex)
$

$
Average\ amount\ of\ likes\ given\ to\ 1\ preson = \frac{100}{500} * 0.25 = 0.05
$

Given that event of giving a like and receiveng a like are independent events 

$
Probability\ of\ getting\ a\ match = Probability\ to\ receive\ a\ like\ given\ that\ amount\ of\ swipes\ is\ limited * Probability\ to\ give\ a\ like\ given\ that\ amount\ of\ 
swipes\ is\ limited
$

$
Probability\ of\ getting\ a\ match = Average\ amount\ of\ mathces\ per\ 1\ potential\ partner = 0.05 * 0.05 = 0.0025
$

$
Average\ amount\ of\ mathces\ with\ 500\ potential\ partners = 0.0025 * 500 = 1.25
$

To confirm those numbers above here is a simulation of a dating app with given parameters:

In [6]:
POPULATION_SIZE = 1000
LAST_F_ID = 500 # amount of F in population, all the people left are M
PROB_OF_F_TO_PICK_M = 0.25 # probaility of F to like M
PROB_OF_M_TO_PICK_F = 0.25 # probaility of M to like F
AMOUNT_OF_SWIPES = 100

In [7]:
profiles = [Profile(_) for _ in range(1, POPULATION_SIZE + 1)]

In [8]:
distribute_likes(profiles, 
                 amount_of_F=LAST_F_ID, 
                 amount_of_swipes=AMOUNT_OF_SWIPES, 
                 F_to_M_prob=PROB_OF_F_TO_PICK_M, 
                 M_to_F_prob=PROB_OF_M_TO_PICK_F
                 )
receive_likes(profiles)
for profile in profiles:
    profile.find_matches()

In [9]:
profiles[0].introduce()

I'm F my id is 1
I like:
M_684, M_856, M_517, M_776, M_862, M_905, M_779, M_823, M_784, M_829, M_533, M_789, M_960, M_834, M_920, M_750, M_624, M_541, M_547, M_802, M_549, M_592, M_594, M_850, M_894
I was liked by:
M_728, M_946, M_564, M_734, M_820, M_993, M_695, M_742, M_828, M_746, M_917, M_749, M_671, M_543, M_970, M_546, M_801, M_676, M_506, M_720, M_638, M_979, M_937
I matched with:

In total I received 23 likes and 0 of those matched


In [10]:
profiles[999].introduce()

I'm M my id is 1000
I like:
F_388, F_220, F_308, F_350, F_224, F_267, F_225, F_397, F_274, F_317, F_233, F_403, F_408, F_78, F_495, F_115, F_202, F_500, F_46, F_463, F_251, F_210, F_212, F_382, F_383
I was liked by:
F_220, F_33, F_266, F_393, F_138, F_225, F_144, F_20, F_485, F_401, F_360, F_149, F_321, F_27, F_406, F_408, F_325, F_156, F_370, F_249, F_212, F_382
I matched with:
F_408, F_220, F_212, F_382, F_225
In total I received 22 likes and 5 of those matched


In [11]:
central_tendencies(profiles)

Average amount of likes and matches per F profile
[24.928  1.198]
Mode of likes of matches per F profile
ModeResult(mode=array([26,  1], dtype=int64), count=array([ 45, 202], dtype=int64))

Average amount of likes and matches per M profile
[25.072  1.198]
Mode of likes of matches per M profile
ModeResult(mode=array([22,  1], dtype=int64), count=array([ 49, 164], dtype=int64))



Given that the general size of population = 1000, experimental values [24.928  1.198] [25.072  1.198] are close to theoretical values 25 1.25

# Case 2

F population \< M population, P(F to give a like) = P(M to give a like)

$
Average\ amount\ of\ likes\ received\ from\ 1\ F = \frac{100}{667} * 0.25 = 0.03748
$

$
Average\ amount\ of\ likes\ received\ from\ 1\ M = \frac{100}{333} * 0.25 = 0.075075
$

Then $Average\ amount\ of\ likes\ received\ from\ 333\ F = 0.03748 * 333 = 12.48$ and $Average\ amount\ of\ likes\ received\ from\ 667\ M = 0.075075 * 667 = 50.075$

$
Average\ amount\ of\ likes\ given\ to\ 1\ F = \frac{100}{333} * 0.25 = 0.075075
$

$
Average\ amount\ of\ likes\ given\ to\ 1\ M = \frac{100}{667} * 0.25 = 0.03748
$

$
Probability\ of\ getting\ a\ match\ by\ a\ F = Average\ amount\ of\ likes\ received\ from\ 1\ F * Average\ amount\ of\ likes\ given\ to\ 1\ F = 0.03748 * 0.075075 = 0.002814
$
Which is the same as $Probability\ of\ getting\ a\ match\ by\ a\ M = 0.002814$

$
Average\ amount\ of\ mathces\ per\ F\ with\ 667\ potential\ partners = 0.002814 * 667 = 1.8769
$

$
Average\ amount\ of\ mathces\ per\ M\ with\ 333\ potential\ partners = 0.002814 * 333 = 0.937
$

In [12]:
POPULATION_SIZE = 1000
LAST_F_ID = 333 # amount of F in population, all the people left are M
PROB_OF_F_TO_PICK_M = 0.25 # probaility of F to like M
PROB_OF_M_TO_PICK_F = 0.25 # probaility of M to like F
AMOUNT_OF_SWIPES = 100

In [13]:
profiles = [Profile(_) for _ in range(1, POPULATION_SIZE + 1)]

In [14]:
distribute_likes(profiles, 
                 amount_of_F=LAST_F_ID, 
                 amount_of_swipes=AMOUNT_OF_SWIPES, 
                 F_to_M_prob=PROB_OF_F_TO_PICK_M, 
                 M_to_F_prob=PROB_OF_M_TO_PICK_F
                 )
receive_likes(profiles)
for profile in profiles:
    profile.find_matches()

In [15]:
profiles[0].introduce()

I'm F my id is 1
I like:
M_992, M_907, M_908, M_782, M_613, M_528, M_911, M_747, M_878, M_836, M_369, M_627, M_457, M_587, M_843, M_419, M_675, M_636, M_384, M_343, M_427, M_981, M_429, M_899, M_772
I was liked by:
M_864, M_993, M_440, M_526, M_738, M_357, M_528, M_697, M_359, M_615, M_913, M_617, M_1000, M_490, M_577, M_492, M_747, M_832, M_453, M_368, M_582, M_499, M_414, M_754, M_628, M_543, M_458, M_969, M_544, M_756, M_799, M_843, M_589, M_504, M_844, M_548, M_763, M_806, M_509, M_722, M_341, M_554, M_937, M_598, M_556, M_514, M_558, M_813, M_898, M_347, M_814, M_645, M_772, M_900, M_390, M_731, M_690, M_902, M_648, M_733, M_693
I matched with:
M_843, M_747, M_772, M_528
In total I received 61 likes and 4 of those matched


In [16]:
profiles[999].introduce()

I'm M my id is 1000
I like:
F_183, F_142, F_1, F_271, F_229, F_59, F_280, F_67, F_238, F_25, F_68, F_29, F_328, F_287, F_288, F_161, F_122, F_255, F_171, F_299, F_131, F_92, F_180, F_138, F_139
I was liked by:
F_258, F_110, F_15, F_27, F_38, F_123, F_124, F_71
I matched with:

In total I received 8 likes and 0 of those matched


In [17]:
central_tendencies(profiles)

Average amount of likes and matches per F profile
[49.9039039   1.87387387]
Mode of likes of matches per F profile
ModeResult(mode=array([51,  2], dtype=int64), count=array([ 27, 100], dtype=int64))

Average amount of likes and matches per M profile
[12.56671664  0.93553223]
Mode of likes of matches per M profile
ModeResult(mode=array([12,  1], dtype=int64), count=array([100, 259], dtype=int64))



Given that the general size of population = 1000, experimental values for F [49.9039039   1.87387387] are close to theoretical values of 50.075 1.8769 and for M [12.56671664  0.93553223] are close to theoretical values 12.48 0.937

# Case 3

F population < M population, P(F to give a like) < P(M to give a like)

$
Average\ amount\ of\ likes\ received\ from\ 1\ F = \frac{100}{667} * 0.14 = 0.021
$

$
Average\ amount\ of\ likes\ received\ from\ 1\ M = \frac{100}{333} * 0.46 = 0.138
$

Then $Average\ amount\ of\ likes\ received\ from\ 333\ F = 0.021 * 333 = 7$ and $Average\ amount\ of\ likes\ received\ from\ 667\ M = 0.138 * 667 = 92.046$

$
Average\ amount\ of\ likes\ given\ to\ 1\ F = \frac{100}{333} * 0.46 = 0.138
$

$
Average\ amount\ of\ likes\ given\ to\ 1\ M = \frac{100}{667} * 0.14 = 0.021
$

$
Probability\ of\ getting\ a\ match\ by\ a\ F = Average\ amount\ of\ likes\ received\ from\ 1\ F * Average\ amount\ of\ likes\ given\ to\ 1\ F = 0.138 * 0.021 = 0.002898
$
Which is the same as $Probability\ of\ getting\ a\ match\ by\ a\ M = 0.002898$

$
Average\ amount\ of\ mathces\ per\ F\ with\ 667\ potential\ partners = 0.002898 * 667 = 1.933
$

$
Average\ amount\ of\ mathces\ per\ M\ with\ 333\ potential\ partners = 0.002898 * 333 = 0.965
$

In [18]:
POPULATION_SIZE = 1000
LAST_F_ID = 333 # amount of F in population, all the people left are M
PROB_OF_F_TO_PICK_M = 0.14 # probaility of F to like M
PROB_OF_M_TO_PICK_F = 0.46 # probaility of M to like F
AMOUNT_OF_SWIPES = 100

In [19]:
profiles = [Profile(_) for _ in range(1, POPULATION_SIZE + 1)]

In [20]:
distribute_likes(profiles, 
                 amount_of_F=LAST_F_ID, 
                 amount_of_swipes=AMOUNT_OF_SWIPES, 
                 F_to_M_prob=PROB_OF_F_TO_PICK_M, 
                 M_to_F_prob=PROB_OF_M_TO_PICK_F
                 )
receive_likes(profiles)
for profile in profiles:
    profile.find_matches()

In [21]:
profiles[0].introduce()

I'm F my id is 1
I like:
M_629, M_427, M_556, M_344, M_524, M_440, M_377, M_687, M_603, M_890, M_593, M_476, M_647, M_723
I was liked by:
M_694, M_696, M_697, M_358, M_530, M_360, M_361, M_701, M_534, M_879, M_540, M_373, M_884, M_545, M_886, M_718, M_889, M_551, M_893, M_894, M_557, M_729, M_902, M_563, M_733, M_905, M_396, M_569, M_910, M_400, M_745, M_576, M_408, M_579, M_410, M_416, M_929, M_762, M_934, M_764, M_765, M_429, M_602, M_433, M_437, M_783, M_444, M_784, M_623, M_453, M_793, M_966, M_796, M_968, M_799, M_970, M_460, M_971, M_972, M_802, M_634, M_804, M_808, M_470, M_812, M_983, M_813, M_817, M_988, M_820, M_821, M_652, M_484, M_658, M_494, M_665, M_495, M_671, M_508, M_682, M_683, M_853, M_855, M_346, M_349, M_520
I matched with:

In total I received 86 likes and 0 of those matched


In [22]:
profiles[999].introduce()

I'm M my id is 1000
I like:
F_140, F_98, F_57, F_100, F_15, F_144, F_103, F_318, F_20, F_191, F_149, F_320, F_235, F_107, F_196, F_111, F_157, F_115, F_223, F_117, F_32, F_34, F_333, F_2, F_4, F_309, F_209, F_252, F_81, F_125, F_253, F_41, F_170, F_42, F_171, F_258, F_174, F_217, F_303, F_91, F_220, F_178, F_137, F_308, F_10, F_181
I was liked by:
F_204, F_243, F_283, F_38, F_230, F_166, F_83
I matched with:

In total I received 7 likes and 0 of those matched


In [23]:
central_tendencies(profiles)

Average amount of likes and matches per F profile
[91.87987988  1.996997  ]
Mode of likes of matches per F profile
ModeResult(mode=array([86,  2], dtype=int64), count=array([20, 96], dtype=int64))

Average amount of likes and matches per M profile
[7.11844078 0.9970015 ]
Mode of likes of matches per M profile
ModeResult(mode=array([7, 1], dtype=int64), count=array([105, 258], dtype=int64))



Given that the general size of population = 1000, experimental values for F [91.87987988  1.996997  ] are close to theoretical values of 92.046 1.933 and for M [7.11844078 0.9970015 ] are close to theoretical values 7 0.965